<a href="https://colab.research.google.com/github/sergmiller/vk-cup-final-2021/blob/main/sergmiller_RuGPT_bundle_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!wget https://storage.yandexcloud.net/vk-cup-data/test_pipeline_questions.csv 

     |████████████████████████████████| 2.6 MB 4.0 MB/s 
     |████████████████████████████████| 895 kB 40.4 MB/s 
     |████████████████████████████████| 3.3 MB 33.3 MB/s 
     |████████████████████████████████| 636 kB 49.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
--2021-08-20 18:02:10--  https://storage.yandexcloud.net/vk-cup-data/test_pipeline_questions.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 250 [text/csv]
Saving to: ‘test_pipeline_questions.csv’

test_pipeline_quest 100%[===================>]     250  --.-KB/s    in 0s      

2021-08-20 18:02:12 (43.2 MB/s) - ‘test_pipeline_questions.csv’ saved [250/250]



In [3]:
import pandas as pd
import numpy as np
from transformers import GPT2LMHeadModel, GPT2Tokenizer


model_name_or_path = "sberbank-ai/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/609 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

In [4]:
def calc_likelihood(text):
    tokens_ids = tokenizer.encode(text, return_tensors="pt").cuda()
    prob = model(tokens_ids)

    tokens = tokens_ids[0].cpu().tolist()
    prob = prob[0].cpu()[0]

    likelyhood = sum([prob[i, j].item() for i, j in enumerate(tokens)])
    return likelyhood

In [5]:
tokens_ids = tokenizer.encode("1 2 3", return_tensors="pt").cuda()
prob = model.forward(tokens_ids, output_hidden_states=True)

In [6]:
logits = prob.logits.cpu().detach().numpy()
hidden = prob.hidden_states
logits.shape, len(hidden), hidden[0].shape

((1, 3, 50257), 25, torch.Size([1, 3, 1536]))

In [7]:
def make_bundle(question: str, answers: list) -> dict:
  torch2np = lambda x: x.cpu().detach().numpy()
  def single_sent_info(sent: str) -> dict:
    tokens_ids = tokenizer.encode(sent, return_tensors="pt").cuda()
    prob = model.forward(tokens_ids,output_hidden_states=True)
    logits = prob.logits[0]
    hidden = prob.hidden_states
    T = len(tokens_ids[0])
    assert list(logits.shape) == [T, 50257], logits.shape
    assert len(hidden) == 25 and list(hidden[-1].shape) == [1, T, 1536], hidden[0].shape
    state = dict()
    state["tokens"] = torch2np(tokens_ids)
    tokens = tokens_ids[0].cpu().tolist()
    logits = torch2np(logits)
    state["likelihood"] = np.sum([logits[i, j] for i, j in enumerate(tokens)])
    state["embed"] = torch2np(hidden[-1])
    return state
  bundle = dict()
  bundle["q"] = single_sent_info(question)
  bundle["a1"] = single_sent_info(question + " " + answers[0])
  bundle["a2"] = single_sent_info(question + " " + answers[1])
  bundle["a3"] = single_sent_info(question + " " + answers[2])
  return bundle

In [8]:
question = "Кто доказал теорию эволюции?"
answers = ["Даня Милохин", "Чарльз Дарвин", "Ленардо Да Винчи"]
def predict(question, answers):
  scores = [calc_likelihood(question + " " +  answer) for answer in answers]
  return str(np.argmax(scores))

# predict(question, answers)
b = make_bundle(question, answers)

In [9]:
b["a1"]["likelihood"]

40.67775

In [11]:
from google.colab import drive
drive.mount('/gdrive')

MOUNT_DIR = "/gdrive/My Drive/Colab Notebooks/ColabMountDir/"
train_data = pd.read_csv(MOUNT_DIR + 'rus_train_dataset.csv', encoding='utf-8', sep='|')
en_ru_aux_data = pd.read_csv(MOUNT_DIR + 'en_to_rus_train_dataset.csv', encoding='utf-8', sep='|')

Mounted at /gdrive


In [13]:
train_data

,Unnamed: 0,question,ps_0,ps_1,ps_2,right_answer_id
0,0,Как называется половина основного времени матч...,Тайм,Период,Гейм,0
1,1,Какая из указанных команд чаще других становил...,Россия,Англия,Чехия,0
2,2,В каком году мужская сборная СССР по баскетбол...,1972,1980,1988,0
3,3,"Как называется технология, используемая для бе...",NFC,Android Pay,Apple Pay,0
4,4,Где в 2014 году проходил Чемпионат мира по фут...,Бразилия,Новая Зеландия,Германия,0
...,...,...,...,...,...,...
4056,4056,"Как называются рецепторы в организме человека,...",Хеморецепторы,Барорецепторы,Механорецепторы,1
4057,4057,Какая из перечисленных стран НЕ входила в сост...,Казахстан,Украина,Финляндия,2
4058,4058,В каком году была выпущена приставка «Playstat...,2000,2001,1999,0
4059,4059,Как называлась добыча мёда диких пчёл на Руси?,Бортничество,Капище,Полюдье,0


In [12]:
en_ru_aux_data

,Unnamed: 0,question,ps_0,ps_1,ps_2,right_answer_id
0,0,Одним из остатков этого государства прежний ст...,Оклахома,Флорида,Луизиана,2.0
1,1,Самое большое озеро в этом штате - озеро окехо...,Гавайи,Теннесси,Флорида,2.0
2,2,Какая государственная столица Вашингтона?,Олимпия,Сиэтл,Walla Walla,0.0
3,3,Какова государственная столица Калифорнии?,Сакраменто,Лос-Анджелес,Сан-Диего,0.0
4,4,Что такое государственная столица Айдахо?,Бойсе,Айдахо-Фолс,Покателло,0.0
...,...,...,...,...,...,...
1936,1948,Три собаки и два щенка весят 128 фунтов. Четыр...,32 фунта,16 фунтов,80 фунтов,2.0
1937,1949,"Изабелла взяла тест, который имел 20 вопросов....",10,5,3,1.0
1938,1950,"Какие две планеты солнечной системы, отличающи...",Меркурий и Марс,Меркурий и Венеры,Венера и Юпитер,1.0
1939,1951,"Эта луна, подшипщая название паромных мертвых ...",Charon,Nereid,Proteus,0.0


In [97]:
for x in train_data.values:
  print(x)
  break

[0 'Как называется половина основного времени матча в футболе?' 'Тайм'
 'Период' 'Гейм' 0]


In [13]:
import tqdm
import json
import os

def prepare_dataset_bundle(df: pd.DataFrame)->list:
  # if not os.path.exists(dirpath):
  #   os.mkdir(dirpath)
  res = []
  for line in tqdm.tqdm(df.values, position=0):
    idx, q, a1, a2, a3, _ = line
    bundle = make_bundle(q, [str(a1), str(a2), str(a3)])
    # np.save(dirpath + "/" + str(idx), bundle)
    res.append(bundle)
  return res

In [77]:
train_bundle = prepare_dataset_bundle(train_data)

100%|██████████| 4061/4061 [16:39<00:00,  4.06it/s]


In [78]:
np.save(MOUNT_DIR + "train_dataset_bundle", train_bundle)

In [14]:
aux_dataset_bundle = prepare_dataset_bundle(en_ru_aux_data)

100%|██████████| 1941/1941 [09:19<00:00,  3.47it/s]


In [16]:
aux_dataset_bundle[-1]

{'a1': {'embed': array([[[-0.10105464,  0.01616081,  0.42873275, ..., -0.10581393,
            1.0124849 , -0.6411731 ],
          [-0.84845036, -0.23344773,  0.09690423, ..., -1.0865364 ,
            0.1056523 ,  0.13522843],
          [-0.71256673, -0.6480566 ,  0.00300177, ..., -1.2034646 ,
           -0.12873265,  0.3060442 ],
          ...,
          [-0.21363114, -0.73018676, -0.47411677, ..., -0.34579372,
            0.42838606,  0.01017577],
          [ 1.08882   , -1.379691  ,  0.4502308 , ..., -0.7921384 ,
            0.85322446, -1.0489603 ],
          [-0.4810411 , -0.10261427, -0.40849409, ..., -0.4148412 ,
            0.6340526 , -0.5959289 ]]], dtype=float32),
  'likelihood': 196.58864,
  'tokens': array([[ 4182,   401, 49176,   334, 35942, 24865, 29316,   665,    16,
          13334,   549,   282,  6292,  1498,  2503, 43016, 40950,    16,
           1586,  1040, 17049, 11138, 38347,   507,   473,  4298,  1748,
            665,    13,   281, 11850,  1683, 29258,    18, 2

In [17]:
np.save(MOUNT_DIR + "en_ru_aux_dataset_bundle", aux_dataset_bundle)

In [ ]:
def create_submission(questions_csv_path):
  data = pd.read_csv('test_pipeline_questions.csv', encoding='utf-8', sep='|')
  answers = [predict(data['question'][i],
                     [data['ps_0'][i], data['ps_1'][i], data['ps_2'][i]]) for i in range(len(data))]
  
  with open('answer.txt', 'w') as f:
    f.write('\n'.join(answers))
  return answers

In [49]:
data = pd.read_csv('test_pipeline_questions.csv', encoding='utf-8', sep='|')

In [50]:
data.head()

,question,ps_0,ps_1,ps_2
0,В каком году был создан ВКонтакте?,1929,2021,2006
1,Кто придумал теорию относительности?,Альберт Энштейн,Исаак Ньютон,Мирон Федоров


Mounted at /gdrive


In [61]:
 data.to_csv("/gdrive/My Drive/Colab Notebooks/ColabMountDir/test_pipeline_questions_comma_sep.csv")